In [1]:
# On importe la librairie bigml et pandas
from bigml.api import BigML
import pandas as pd
import pprint

In [2]:
# On se connecte à bigml et on récupère un ensemble et son dataset
api = BigML(project='project/5d94a32e42129f2e16000232')
ensemble = api.get_ensemble('ensemble/5dc2f3f07811dd7f2d012312')
dataset = api.get_dataset(ensemble['object']['dataset'])

In [3]:
# Data to predict
predicton_data = {
    "RevolvingUtilizationOfUnsecuredLines": 0.01703559,
    "NumberOfDependents": 1,
    "DebtRatio": 0,
    "age": 42,
    "NumberOfOpenCreditLinesAndLoans": 6,
    "NumberRealEstateLoansOrLines": 15,
    "NumberOfTime30-59DaysPastDueNotWorse": 15,
    "NumberOfTime60-89DaysPastDueNotWorse": 15,
    "NumberOfTimes90DaysLate": 0
}

In [4]:
# On récupère l'ensemble des colonnes du dataset
dataset_columns = []
for row in dataset['object']['fields']:
    dataset_columns.append(dataset['object']['fields'][row]['name'])

In [5]:
# On crée un dataframe pandas et on injecte les colonnes
df = pd.DataFrame(columns=dataset_columns)
# puis les données
df = df.append(predicton_data,ignore_index=True)

In [6]:
# Traitement du dataset
df[['NumberOfDependents']] = df[['NumberOfDependents']].fillna(0)
df[['MonthlyIncome']] = df[['MonthlyIncome']].fillna(0)

df.loc[df['NumberOfTime30-59DaysPastDueNotWorse'] >= 20,'NumberOfTime30-59DaysPastDueNotWorse'] = df['NumberOfTime30-59DaysPastDueNotWorse'].median()
df.loc[df['NumberOfTime60-89DaysPastDueNotWorse'] >= 20,'NumberOfTime60-89DaysPastDueNotWorse'] = df['NumberOfTime60-89DaysPastDueNotWorse'].median()
df.loc[df['NumberOfTimes90DaysLate'] >= 20,'NumberOfTimes90DaysLate'] = df['NumberOfTimes90DaysLate'].median()

df['NumberOfTimeGlobal'] = df['NumberOfTime30-59DaysPastDueNotWorse'] + df['NumberOfTime60-89DaysPastDueNotWorse']*2 + df['NumberOfTimes90DaysLate']*3
df.drop(['NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfTimes90DaysLate'], axis=1, inplace=True)

df.loc[df['age'] >= 70,'IsOld'] = 1
df.loc[df['age'] < 70,'IsOld'] = 0

In [7]:
# Téléchargement du datatset
df.to_csv('storage/dataset_one_line_prediction.csv', index=False) 

In [8]:
# Création de la source et du dataset sur BigML
source_one_line = api.create_source('storage/dataset_one_line_prediction.csv')
dataset_one_line = api.create_dataset(source_one_line, {"name": "Dataset One Line"})
api.ok(dataset_one_line)

True

In [9]:
import csv
from bigml.ensemble import Ensemble
local_ensemble = Ensemble('ensemble/5dc2f3f07811dd7f2d012312')
with open("storage/batch_one_prediction.csv") as test_handler:
    reader = csv.DictReader(test_handler)
    for input_data in reader:
    # predicting for all rows
        results = local_ensemble.predict(input_data)
        print(results)

{'probability': 0.50998, 'prediction': '0'}
